<a href="https://colab.research.google.com/github/pjot726/PyTorchTutorials/blob/main/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

Here, we import needed packages from PyTorch. torchvision.datasets contains the datasets of interest, and DataLoader wraps an iterable around the dataset.

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

We now download the training and test data from the 